In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import grid_search
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2641: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [2]:
train = pd.read_json('../data/train.json/train.json')
train.head()

clean_string = [' , '.join(z).strip() for z in train['ingredients']]  
ingredients_string = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in train['ingredients']]

cnt_vec = CountVectorizer(max_features = 2000)
tfidf_vec = TfidfVectorizer(stop_words='english',
                             ngram_range = ( 1 , 1 ),
                             analyzer="word", 
                             max_df = .57 , 
                             binary=False , 
                             token_pattern=r'\w+', 
                             sublinear_tf=False)

comb_feats = FeatureUnion([("cnt", cnt_vec), ("tfidf", tfidf_vec)])

ingredients = train['ingredients']
words_list = [' '.join(x) for x in ingredients]

###test
words_list = ingredients_string

#Make label encoder
le = preprocessing.LabelEncoder()
le.fit(train["cuisine"])

labels = le.transform(train['cuisine'])

In [3]:
'''
bag_of_words = vectorizer.fit(words_list)
bag_of_words = vectorizer.transform(words_list).toarray()
print(bag_of_words.shape)

tfidf_feat = vectorizertr.fit(words_list)
tfidf_feat = vectorizertr.transform(words_list).toarray()
'''
X_features = comb_feats.fit(words_list, labels).transform(words_list)

In [4]:
print type(X_features)
print X_features.shape
print train['cuisine'].shape

<class 'scipy.sparse.csr.csr_matrix'>
(39774, 4963)
(39774,)


In [16]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

batch_size = 128
nb_epoch = 15

X_train = X_features.todense()
y_train = le.transform(train["cuisine"])

nb_classes = len(le.classes_)

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)

#output_dim=64, input_dim=100,
model = Sequential()
model.add(Dense(1024,input_shape=(4963,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(.4))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(.4))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

rms = RMSprop()
model.compile(loss='categorical_crossentropy', optimizer=rms)

model.fit(X_train, Y_train,
          batch_size=batch_size, nb_epoch=nb_epoch,
          show_accuracy=True, verbose=2)

Epoch 1/15
3s - loss: 1.2840 - acc: 0.6226
Epoch 2/15
3s - loss: 0.8038 - acc: 0.7635
Epoch 3/15
3s - loss: 0.6430 - acc: 0.8128
Epoch 4/15
3s - loss: 0.5183 - acc: 0.8470
Epoch 5/15
3s - loss: 0.4131 - acc: 0.8772
Epoch 6/15
3s - loss: 0.3272 - acc: 0.9025
Epoch 7/15
3s - loss: 0.2579 - acc: 0.9226
Epoch 8/15
3s - loss: 0.1981 - acc: 0.9397
Epoch 9/15
3s - loss: 0.1582 - acc: 0.9512
Epoch 10/15
3s - loss: 0.1298 - acc: 0.9603
Epoch 11/15
3s - loss: 0.1084 - acc: 0.9656
Epoch 12/15
3s - loss: 0.0963 - acc: 0.9706
Epoch 13/15
3s - loss: 0.0814 - acc: 0.9749
Epoch 14/15
3s - loss: 0.0693 - acc: 0.9783
Epoch 15/15
3s - loss: 0.0638 - acc: 0.9806


In [17]:
#Now read the test json file in 
test = pd.read_json('../data/test.json/test.json')
test.head()

#Do the same thing we did with the training set and create a array using the count vectorizer. 
test_ingredients = test['ingredients']
test_ingredients_words = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in test['ingredients']]

In [18]:
X_test = comb_feats.transform(test_ingredients_words).todense()
result = model.predict_classes(X_test)

9944/9944 [==============================] - 0s     


In [19]:
result = le.inverse_transform(result)
output = pd.DataFrame( data={"id":test["id"], "cuisine":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "ANN_featunion_model.csv", index=False, quoting=3 )

In [ ]:
traindf = pd.read_json("../data/train.json/train.json")
traindf['ingredients_clean_string'] = [' , '.join(z).strip() for z in traindf['ingredients']]  
traindf['ingredients_string'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in traindf['ingredients']]       

tmp = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in traindf['ingredients']]
print type(tmp)

In [ ]:
print tmp[0]

In [ ]:
# A combination of Word lemmatization + LinearSVC model finally pushes the accuracy score past 80%





corpustr = traindf['ingredients_string']
vectorizertr = TfidfVectorizer(stop_words='english',
                             ngram_range = ( 1 , 1 ),analyzer="word", 
                             max_df = .57 , binary=False , token_pattern=r'\w+' , sublinear_tf=False)
tfidftr=vectorizertr.fit_transform(corpustr).todense()
corpusts = testdf['ingredients_string']
vectorizerts = TfidfVectorizer(stop_words='english')
tfidfts=vectorizertr.transform(corpusts)

predictors_tr = tfidftr

targets_tr = traindf['cuisine']

predictors_ts = tfidfts


#classifier = LinearSVC(C=0.80, penalty="l2", dual=False)
parameters = {'C':[1, 10]}
#clf = LinearSVC()
clf = LogisticRegression()

classifier = grid_search.GridSearchCV(clf, parameters)

classifier=clf.fit(predictors_tr,targets_tr)

testdf = pd.read_json("../data/test.json/test.json") 
testdf['ingredients_clean_string'] = [' , '.join(z).strip() for z in testdf['ingredients']]
testdf['ingredients_string'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in testdf['ingredients']]       


predictions=classifier.predict(predictors_ts)
testdf['cuisine'] = predictions
testdf = testdf.sort('id' , ascending=True)

#testdf[['id' , 'ingredients_clean_string' , 'cuisine' ]].to_csv("submission.csv")

result = predictions

output = pd.DataFrame( data={"id":testdf["id"], "cuisine":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "test_model.csv", index=False, quoting=3 )